# Chapter 09 code snippets
This notebook contains all code snippets from chapter 9.

## Training simple sklearn model

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
diabetes_ds = ws.datasets['diabetes']

training_data, validation_data = diabetes_ds.random_split(percentage = 0.8)

X_train = training_data.drop_columns('target').to_pandas_dataframe()
y_train = training_data.keep_columns('target').to_pandas_dataframe()

X_validate = validation_data.drop_columns('target').to_pandas_dataframe()
y_validate = validation_data.keep_columns('target').to_pandas_dataframe()

In [ ]:
from sklearn.linear_model import LassoLars

alpha = 0.1

model = LassoLars(alpha=alpha)
model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error

predictions = model.predict(X_validate)
rmse = mean_squared_error(predictions, y_validate, squared = False)

# Range of data using the peak to peak numpy function
range_y_validate = y_validate.to_numpy().ptp()
# Normalize dividing by the range of the data
nrmse = rmse/range_y_validate

print(f"Normalized Root Mean Squared Error: {nrmse}")

In [ ]:
import os
import joblib

os.makedirs('./outputs', exist_ok=True)

model_file_name = f'model_{nrmse:.4f}_{alpha:.4f}.pkl'
joblib.dump(value=model, filename=os.path.join('./outputs/',model_file_name))

## Tracking metrics in experiments

In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="chapter09")

In [ ]:
run = exp.start_logging()
print(run.get_details())

In [ ]:
run.log("nrmse", 0.01)
run.log(name="nrmse", value=0.015, description="2nd measure")

In [ ]:
run.log_list("accuracies", [0.5, 0.57, 0.62])

In [ ]:
run.log_table("table", {"x":[1, 2], "y":[0.1, 0.2]})
run.log_row("table", x=3, y=0.3)

In [ ]:
child_run = run.child_run()
child_run.log("child_metric", 0.01)
child_run.parent.log("metric_from_child", 0.02)

In [ ]:
child_run.cancel()
run.complete()
print(run.get_status())

### Tracking model evolution

In [ ]:
import shutil

try:
  shutil.rmtree("./outputs")
except FileNotFoundError: 
  pass

In [ ]:
from sklearn.linear_model import LassoLars
from sklearn.metrics import mean_squared_error

def train_and_evaluate(alpha, X_t, y_t, X_v, y_v):
  model = LassoLars(alpha=alpha)
  model.fit(X_t, y_t)
  predictions = model.predict(X_v)
  rmse = mean_squared_error(predictions, y_v, squared = False)
  range_y_validate = y_v.to_numpy().ptp()
  nrmse = rmse/range_y_validate
  print(f"NRMSE: {nrmse}")
  return model, nrmse

trained_model, model_nrmse = train_and_evaluate(0.1, 
                        X_train, y_train,
                        X_validate, y_validate)

In [ ]:
def train_and_evaluate(run, alpha, X_t, y_t, X_v, y_v):
  model = LassoLars(alpha=alpha)
  model.fit(X_t, y_t)
  predictions = model.predict(X_v)
  rmse = mean_squared_error(predictions, y_v, squared = False)
  range_y_validate = y_v.to_numpy().ptp()
  nrmse = rmse/range_y_validate
  run.log("nrmse", nrmse)
  run.log_row("nrmse over α", α=alpha, nrmse=nrmse)
  return model, nrmse

In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="chapter09")
with exp.start_logging() as run:
    print(run.get_portal_url())
    for a in [0.001, 0.01, 0.1, 0.25, 0.5]:
        train_and_evaluate(run, a, 
                            X_train, y_train,
                            X_validate, y_validate)

### Using MLFlow to track experiments

In [ ]:
from sklearn.linear_model import LassoLars
from sklearn.metrics import mean_squared_error
import mlflow

def train_and_evaluate(alpha, X_t, y_t, X_v, y_v):
  model = LassoLars(alpha=alpha)
  model.fit(X_t, y_t)
  predictions = model.predict(X_v)
  rmse = mean_squared_error(predictions, y_v, squared = False)
  range_y_validate = y_v.to_numpy().ptp()
  nrmse = rmse/range_y_validate
  mlflow.log_metric("nrmse", nrmse)
  return model, nrmse

mlflow.set_experiment("chapter09-mlflow")
with mlflow.start_run():
    mlflow.sklearn.autolog()
    trained_model, model_nrmse = train_and_evaluate(0.1, 
                                    X_train, y_train,
                                    X_validate, y_validate)

You can open the MLflow ui by posting the `mlflow ui` command in a terminal.
This will open a web service listening at port 5000 of your compute instance.
To view that port you need to visit `https://<compute-instance-name>-5000.<location>.instances.azureml.ms/`.

If you are having troubles constructing this url:
- Open a JupyterLab. 
- Copy the url that should be like `https://<compute-instance-name>.<location>.instances.azureml.ms`.
- Modify the url to include the 5000 port number.

In [ ]:
!pip install azureml-mlflow

In [ ]:
import mlflow
from azureml.core import Workspace

ws = Workspace.from_config()
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

mlflow.set_experiment("chapter09-mlflow")
with mlflow.start_run():
    mlflow.sklearn.autolog()
    trained_model, model_nrmse = train_and_evaluate(0.1, 
                                    X_train, y_train,
                                    X_validate, y_validate)

## Scaling the training process with compute clusters

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core import ScriptRunConfig

ws = Workspace.from_config()
target = ws.compute_targets['cpu-sm-cluster']

script = ScriptRunConfig(
    source_directory='greeter-job',
    script='greeter.py',
    compute_target=target,
    arguments=['--greet-name', 'packt']
)

exp = Experiment(ws, 'greet-packt')
run = exp.submit(script)
print(run.get_portal_url())
run.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails

run = exp.submit(script)
RunDetails(run).show()

### Understanding execution environments

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()

In [ ]:
from azureml.core import Environment

minimal_env = Environment.get(ws, name="AzureML-Minimal")

print(minimal_env.name, minimal_env.version)
print(minimal_env.python.conda_dependencies.serialize_to_string())

In [ ]:
from azureml.core import Experiment, ScriptRunConfig

target = ws.compute_targets['cpu-sm-cluster']

script = ScriptRunConfig(
    source_directory='greeter-job',
    script='greeter.py',
    environment=minimal_env,
    compute_target=target,
    arguments=['--greet-name', 'packt']
)

exp = Experiment(ws, 'greet-packt')
run = exp.submit(script)
print(run.get_portal_url())
run.wait_for_completion(show_output=True)

In [ ]:
from azureml.core import Environment

banner_env = Environment.from_conda_specification(
                            name = "banner-env",
                            file_path = "greeter-banner-job.yml")
banner_env.environment_variables["GREET_HEADER"] = \
                                 "Env. var. header:"

In [ ]:
# To register in the workspace, uncomment the following line
# banner_env.register(ws)

In [ ]:
script = ScriptRunConfig(
    source_directory='greeter-banner-job',
    script='greeter.py',
    environment=banner_env,
    compute_target=target,
    arguments=['--greet-name', 'packt']
)

exp = Experiment(ws, 'greet-packt')
run = exp.submit(script)
print(run.get_portal_url())
run.wait_for_completion(show_output=True)

### Training the diabetes model on a compute cluster

In [ ]:
!pip show scikit-learn

In [ ]:
import sklearn
print(sklearn.__version__)

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies 
import sklearn

diabetes_env = Environment(name="diabetes-training-env")
diabetes_env.python.conda_dependencies = CondaDependencies()
diabetes_env.python.conda_dependencies.add_conda_package(f"scikit-learn=={sklearn.__version__}")
# Optionally 
# import joblib
# diabetes_env.python.conda_dependencies.add_pip_package(f"joblib=={joblib.__version__}")
# Alternatively you can use the following single line code:
# diabetes_env.python.conda_dependencies = CondaDependencies.create(
#               conda_packages=[f"scikit-learn=={sklearn.__version__}"],
#               pip_packages=[f"joblib=={joblib.__version__}"])

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core import ScriptRunConfig

ws = Workspace.from_config()
target = ws.compute_targets['cpu-sm-cluster']

script = ScriptRunConfig(
    source_directory='diabetes-training',
    script='training.py',
    environment=diabetes_env,
    compute_target=target,
    arguments=['--alpha', 0.01]
)

exp = Experiment(ws, 'chapter09-diabetes')
run = exp.submit(script)
run.wait_for_completion(show_output=True)

## Hyperparameter tuning using HyperDrive
Make sure you have executed the previous section.

In [ ]:
from azureml.train.hyperdrive import HyperDriveConfig
from azureml.train.hyperdrive import RandomParameterSampling, uniform, PrimaryMetricGoal

param_sampling = RandomParameterSampling({
        'alpha': uniform(0.00001, 0.1),
    }
)

script = ScriptRunConfig(
    source_directory='diabetes-training',
    script='training.py',
    environment=diabetes_env,
    compute_target=target
)

hd_config = HyperDriveConfig(
                     run_config=script,                          
                     hyperparameter_sampling=param_sampling,
                     primary_metric_name="nrmse", 
                     primary_metric_goal=                   
                                 PrimaryMetricGoal.MINIMIZE,
                     max_total_runs=20,
                     max_concurrent_runs=4)

experiment = Experiment(ws, "chapter09-hyperdrive")
hyperdrive_run = experiment.submit(hd_config)

hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics(name='nrmse')
parameter_values = best_run.get_details()[
                        'runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('- NRMSE:', best_run_metrics['nrmse'])
print('- alpha:', parameter_values[1])

### Using the early termination policy

In [ ]:
from azureml.core import Workspace, ScriptRunConfig, Experiment
from azureml.train.hyperdrive import (
    GridParameterSampling,    
    choice,
    MedianStoppingPolicy,
    HyperDriveConfig,
    PrimaryMetricGoal
)

ws = Workspace.from_config()
target = ws.compute_targets["cpu-sm-cluster"]

script = ScriptRunConfig(
    source_directory="termination-policy-training",
    script="training.py",
    environment=ws.environments["AzureML-Minimal"],
    compute_target=target,
)

param_sampling = GridParameterSampling(
    {
        "a": choice(6, 5, 4, 3, 2, 1),
        "b": choice(6, 5, 4, 3, 2, 1),
    }
)

early_termination_policy = MedianStoppingPolicy(
    evaluation_interval=1, delay_evaluation=5
)

# More aggressive alternative
# from azureml.train.hyperdrive import TruncationSelectionPolicy
# early_termination_policy = TruncationSelectionPolicy(
#    truncation_percentage=50, evaluation_interval=1
#)

hd_config = HyperDriveConfig(
    policy=early_termination_policy,
    run_config=script,
    hyperparameter_sampling=param_sampling,
    primary_metric_name="fake_metric",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=50,
    max_concurrent_runs=4
)

experiment = Experiment(ws, "chapter09-hyperdrive")
hyperdrive_run = experiment.submit(hd_config)

hyperdrive_run.wait_for_completion(show_output=True)

## Running AutoML experiments via code

In [ ]:
from azureml.core import Workspace, Dataset
from azureml.train.automl import AutoMLConfig

ws = Workspace.from_config()
compute_target = ws.compute_targets["cpu-sm-cluster"]

diabetes_dataset = Dataset.get_by_name(workspace=ws, name='diabetes')

train_ds,validate_ds = diabetes_dataset.random_split(percentage=0.8, seed=1337)

print(f"Train: {len(train_ds.to_pandas_dataframe().axes[0])} rows")
print(f"Validate: {len(validate_ds.to_pandas_dataframe().axes[0])} rows")

experiment_config = AutoMLConfig(
    task = "regression",
    primary_metric = 'normalized_root_mean_squared_error',
    training_data = train_ds,
    label_column_name = "target",
    validation_data = validate_ds,
    compute_target = compute_target,
    experiment_timeout_hours = 0.25,
    iterations = 4
)

In [ ]:
from azureml.core.experiment import Experiment

my_experiment = Experiment(ws, 'chapter09-automl-experiment')
run = my_experiment.submit(experiment_config, show_output=True)

In [ ]:
# To retrieve a previous run 
# from azureml.train.automl.run import AutoMLRun
# run = AutoMLRun(my_experiment, run_id = 'AutoML_80833402-6e7e-4c25-b6aa-b6fd44d75d09')

# Get best run and model
best_run, best_model = run.get_output()
# Or with index
best_run = run.get_output()[0]
best_model = run.get_output()[1]


In [ ]:
# Scaling/normalization and model algorythm
best_model.steps

In [ ]:
# Automated feature engineering
# Use 'timeseriestransformer' for task='forecasting',
# else use 'datatransformer' for 'regression' or 'classification' task.
print(best_model.named_steps['datatransformer'] \
                 .get_featurization_summary())
feature_names=best_model.named_steps['datatransformer'] \
                 .get_engineered_feature_names()
print("Engineered feature names:")
print(feature_names)

## Working with pipelines